In [140]:
import polars as pl
import random
import uuid
import datetime
# from IPython.display import display, clear_output

# Read the base data

In [66]:
pat_df = pl.read_excel('ai-acc-sys-dummy-data.xlsx',sheet_name='JE_patterns')

In [67]:
pat_df.head()

Pattern,Dr_id,Dr_name,Cr_id,Cr_name,件数,金額係数,PL_大カテゴリ
i64,i64,str,i64,str,i64,f64,str
1,11000,"""売掛金""",61000,"""売上""",500,1.0,"""売上"""
2,12000,"""棚卸資産""",21000,"""買掛金""",500,0.5,"""売上原価"""
3,51000,"""売上原価""",12000,"""棚卸資産""",500,0.5,"""売上原価"""
4,14000,"""現金預金""",11000,"""売掛金""",50,2.0,"""N/A"""
5,21000,"""買掛金""",14000,"""現金預金""",50,1.0,"""N/A"""


In [68]:
pat_df.describe()

describe,Pattern,Dr_id,Dr_name,Cr_id,Cr_name,件数,金額係数,PL_大カテゴリ
str,f64,f64,str,f64,str,f64,f64,str
"""count""",40.0,40.0,"""40""",40.0,"""40""",40.0,40.0,"""40"""
"""null_count""",0.0,0.0,"""0""",0.0,"""0""",0.0,0.0,"""0"""
"""mean""",20.5,42860.3,null,27025.375,null,44.5,0.43,null
"""std""",11.690452,16085.224233,null,15074.19624,null,131.728158,0.397557,null
"""min""",1.0,11000.0,"""リース料""",11000.0,"""受取利息""",5.0,0.3,"""N/A"""
"""max""",40.0,52028.0,"""雑給""",62005.0,"""買掛金""",500.0,2.0,"""販管費"""
"""median""",20.5,52008.5,null,22000.0,null,5.0,0.3,null
"""25%""",11.0,51000.0,null,22000.0,null,5.0,0.3,null
"""75%""",31.0,52019.0,null,22000.0,null,5.0,0.3,null


In [69]:
# pat_df.with_columns(pl.col(['Dr_name','Cr_name','PL_大カテゴリ']).apply(lambda x: x.replace('"',''))) # tried to remove double quotes but did not work. mayble this is just a renderting thing?

In [70]:
acc_df = pl.read_excel('ai-acc-sys-dummy-data.xlsx',sheet_name='Account_master')
acc_df.head()

account_id,account_name,category
i64,str,str
11000,"""売掛金""","""Asset"""
12000,"""棚卸資産""","""Asset"""
13000,"""仮払消費税""","""Asset"""
14000,"""現金預金""","""Asset"""
15000,"""未収収益""","""Asset"""


In [71]:
emp_df = pl.read_excel('ai-acc-sys-dummy-data.xlsx',sheet_name='employee_master')

In [118]:
emp_df.head()

employee_id,employee_name,l,former_postion,join_date,department,Role
i64,str,str,str,str,str,str
20230001,"""小倉 史郎""","""Chief Financia…","""Controller""","""04-01-10""","""Accounting""","""Manager"""
20230002,"""幾田 宏美""","""Controller""","""Accounting Man…","""12-01-15""","""Accounting""","""Approver"""
20230003,"""島田 幸司""","""Accounting Man…","""Accountant""","""03-01-12""","""Accounting""","""Approver"""
20230004,"""木原 圭一""","""Accounting Man…","""Accountant""","""05-01-13""","""Accounting""","""Approver"""
20230005,"""水尾 洋介""","""Accounting Man…","""N/A""","""04-01-17""","""Accounting""","""Approver"""


In [73]:
# Define the logic of creating dummy data out of the dataframs above

## Iterate over the patterns of the jounal entries for the number of times specified in "件数" column
## The VAT(comsunption tax) is not considered. Maybe add it later on

# Align the needed snippets of the dataframes

In [133]:
emp_list = emp_df.select(pl.col('employee_name')).to_series().to_list() # selectからの型変換の方法チェック
print(emp_list)

['小倉 史郎', '幾田 宏美', '島田 幸司', '木原 圭一', '水尾 洋介', '大西 明久', '小森 祥子', '宮脇 謙吾', '植松 昭仁', '齋藤 直子', '佐藤 弘樹', '白尾 弘美', '黒川 摩耶', '中原 彩', '近藤 賢治', '亀本 信子', '榎本 善幸', '片岡 瑛子', '岡 光二', '河村 大志']


In [97]:
random.choice(emp_list)

'小倉 史郎'

ふむふむ、なるほどね...

In [104]:
creator_df = emp_df.filter(pl.col('Role')=='Create')
creator_df.head(3)

employee_id,employee_name,l,former_postion,join_date,department,Role
i64,str,str,str,str,str,str
20230007,"""小森 祥子""","""Accountant""","""N/A""","""04-01-19""","""Accounting""","""Create"""
20230008,"""宮脇 謙吾""","""Accountant""","""N/A""","""04-01-22""","""Accounting""","""Create"""
20230009,"""植松 昭仁""","""Accountant""","""N/A""","""02-01-20""","""Accounting""","""Create"""


In [105]:
approver_df = emp_df.filter(pl.col('Role')=='Approver')
approver_df.head(3)

employee_id,employee_name,l,former_postion,join_date,department,Role
i64,str,str,str,str,str,str
20230002,"""幾田 宏美""","""Controller""","""Accounting Man…","""12-01-15""","""Accounting""","""Approver"""
20230003,"""島田 幸司""","""Accounting Man…","""Accountant""","""03-01-12""","""Accounting""","""Approver"""
20230004,"""木原 圭一""","""Accounting Man…","""Accountant""","""05-01-13""","""Accounting""","""Approver"""


In [139]:
creator_ids = creator_df.select(pl.col('employee_id')).to_series().to_list()
approver_ids = approver_df.select(pl.col('employee_id')).to_series().to_list()
print(creator_ids)
print(approver_ids)

[20230007, 20230008, 20230009, 20230010, 20230011, 20230012, 20230013, 20230014, 20230015, 20230016, 20230017, 20230018, 20230019, 20230020]
[20230002, 20230003, 20230004, 20230005, 20230006]


# Create dummy records by looping

In [94]:
pat_df.head(1) # shemaのチェック用

Pattern,Dr_id,Dr_name,Cr_id,Cr_name,件数,金額係数,PL_大カテゴリ
i64,i64,str,i64,str,i64,f64,str
1,11000,"""売掛金""",61000,"""売上""",500,1.0,"""売上"""


In [163]:
unit_amount = 1000
record_id = 1
gl_data = []

for row in pat_df.iter_rows(named=True):
    '''Fixed in both Dr. and Cr.'''
    for i in range(row['件数']):
        journal_id = str(uuid.uuid4())
        creator_id = random.choice(creator_ids)
        approver_id = random.choice(approver_ids)
        create_date = datetime.datetime(year=2023,
                                        month=random.choice(range(1,13)),
                                        day=random.choice(range(1,25))
                                       )
        approve_date = create_date + datetime.timedelta(days=random.choice(range(0,6)))
        vat_id = 0 # とりあえず全部課税取引にしておく。
        amount = unit_amount * random.choice(range(1,20)) * (row['金額係数']*10) # 小数が発生した面倒なので金額係数はx10する。
        for acc in ['Dr_id','Cr_id']:
            '''Varied between Dr. and Cr.'''
            record_id = record_id + 1
            account_id = row[acc]
            account_name = acc_df.filter(pl.col('account_id')==account_id) \
                                .select('account_name') \
                                .item()
            subaccount_id = account_id # とりあえずAccount nameと同じにしておく
            debter_creditor = 'Dr.' if acc == 'Dr_id' else 'Cr.'
            amount = amount * (1 if acc == 'Dr_id' else -1) # change depending on dr/cr
            # add the record to the GL list
            gl_data.append({
                            'record_id': record_id,
                            'journal_id': journal_id,
                            'account_id': account_id,
                            'account_name': account_name,
                            'subaccount_id': subaccount_id,
                            'vat_id': vat_id,
                            'debter_creditor': debter_creditor,
                            'amount': amount,
                            'creator': creator_id,
                            'approver': approver_id,
                            'create_date': create_date,
                            'approve_date': approve_date
                            })

In [165]:
gl_data = pl.DataFrame(gl_data)
gl_data.head()

record_id,journal_id,account_id,account_name,subaccount_id,vat_id,debter_creditor,amount,creator,approver,create_date,approve_date
i64,str,i64,str,i64,i64,str,f64,i64,i64,datetime[μs],datetime[μs]
2,"""5afbe1df-7274-…",11000,"""売掛金""",11000,0,"""Dr.""",30000.0,20230013,20230004,2023-04-19 00:00:00,2023-04-24 00:00:00
3,"""5afbe1df-7274-…",61000,"""売上""",61000,0,"""Cr.""",-30000.0,20230013,20230004,2023-04-19 00:00:00,2023-04-24 00:00:00
4,"""fe5a924d-8bb0-…",11000,"""売掛金""",11000,0,"""Dr.""",150000.0,20230017,20230005,2023-11-19 00:00:00,2023-11-24 00:00:00
5,"""fe5a924d-8bb0-…",61000,"""売上""",61000,0,"""Cr.""",-150000.0,20230017,20230005,2023-11-19 00:00:00,2023-11-24 00:00:00
6,"""2e6b2d85-189b-…",11000,"""売掛金""",11000,0,"""Dr.""",150000.0,20230009,20230003,2023-05-19 00:00:00,2023-05-22 00:00:00


# Save the created data as csv

In [168]:
gl_data.write_excel('GL_init_data.xlsx', worksheet='GL_data')